# Yelp Restaurant Photo Classification

https://www.kaggle.com/c/yelp-restaurant-photo-classification

## Загружаем данные

In [1]:
!pip install keras --upgrade
!pip install keras_applications --upgrade
!pip install kaggle


    100% |████████████████████████████████| 307kB 5.7MB/s 
    100% |████████████████████████████████| 51kB 10.1MB/s 
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6
Requirement already up-to-date: keras_applications in /usr/local/lib/python3.6/dist-packages (1.0.4)
    100% |████████████████████████████████| 51kB 2.1MB/s 
    100% |████████████████████████████████| 51kB 5.2MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/3b/8e/14/adcbb71d126f5fb939fec8aa81394450922c20eee876bf8882
Successfully built kaggle


In [0]:
!mkdir ~/.kaggle
!echo '{"username":"11111","key":"111111"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle config set -n path -v /content
!kaggle competitions download -c yelp-restaurant-photo-classification

path is now set to: /content
  0%|                                               | 0.00/1.12M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 1.12M/1.12M [00:00<00:00, 91.0MB/s]
  0%|                                               | 0.00/7.12k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 7.12k/7.12k [00:00<00:00, 9.02MB/s]
 37%|██████████████▍                        | 2.43G/6.54G [00:17<00:36, 123MB/s]

100%|██████████████████████████████████████▉| 6.54G/6.54G [00:49<00:00, 126MB/s]
100%|███████████████████████████████████████| 6.54G/6.54G [00:49<00:00, 142MB/s]
 65%|█████████████████████████▍             | 4.32G/6.61G [00:32<00:23, 105MB/s]

100%|██████████████████████████████████████▉| 6.61G/6.61G [00:48<00:00, 151MB/s]
100%|███████████████████████████████████████| 6.61G/6.61G [00:49<00:00, 145MB/s]
  0%|                                               | 0.00/39.0k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 39.0k/39.0k [00:00<00:00, 33.8MB/s]
  0%|                                               | 0.00/4.79M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 4.79M/4.79M [00:00<00:00, 88.1MB/s]


In [3]:
!ls ./competitions/yelp-restaurant-photo-classification/ -l

total 13804260
-rw-r--r-- 1 root root      39917 Jul 29 12:17 sample_submission.csv.tgz
-rw-r--r-- 1 root root 7102584231 Jul 29 12:17 test_photos.tgz
-rw-r--r-- 1 root root    5019061 Jul 29 12:17 test_photo_to_biz.csv.tgz
-rw-r--r-- 1 root root       7287 Jul 29 12:15 train.csv.tgz
-rw-r--r-- 1 root root 7026724672 Jul 29 12:16 train_photos.tgz
-rw-r--r-- 1 root root    1171455 Jul 29 12:15 train_photo_to_biz_ids.csv.tgz


In [0]:
!tar -xzf ./competitions/yelp-restaurant-photo-classification/test_photos.tgz 
!tar -xzf ./competitions/yelp-restaurant-photo-classification/train_photos.tgz 
!tar -xzf ./competitions/yelp-restaurant-photo-classification/train.csv.tgz 
!tar -xzf ./competitions/yelp-restaurant-photo-classification/train_photo_to_biz_ids.csv.tgz 
!tar -xzf ./competitions/yelp-restaurant-photo-classification/test_photo_to_biz.csv.tgz 


In [0]:
import os
import cv2
import numpy as np
import pandas as pd
#from tensorflow.contrib import keras

import keras
from keras.models import Model, Sequential
#from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
from keras.layers import Dense, Dropout, Flatten, Concatenate
import keras.backend as K
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
#preprocess_input = keras.applications.vgg16.preprocess_input
#VGG16 = keras.applications.VGG16

IMG_SIZE = 224


## Загружаем разметку

In [0]:
train_biz_df = pd.read_csv('train.csv')
train_photos_df = pd.read_csv('train_photo_to_biz_ids.csv')
df = train_photos_df.merge(train_biz_df)

test_photos_df = pd.read_csv('test_photo_to_biz.csv')

In [0]:
def get_image(path, img_id, img_size=(IMG_SIZE, IMG_SIZE)):
    img = cv2.imread(os.path.join(path, '%s.jpg' % img_id))[:,:,::-1]
    img = preprocess_input(img.astype(np.float32))
    return cv2.resize(img, img_size)

def to_dense(labels):
    result = [0] * 9
    for i in labels:
        result[i] = 1.
    return result

def train_generator(df, img_size=(IMG_SIZE, IMG_SIZE), batch_size=32):
    while True:
        df = df.sample(frac=1).reset_index(drop=True)
        for i in range(0, len(df) // batch_size * batch_size, batch_size):
            X, y = [], []
            for _, row in df[i:i + batch_size].iterrows():
                X.append(get_image('train_photos', row['photo_id'], img_size))
                y.append(to_dense(map(int, str(row['labels']).split())))
            y = np.array(y)
            yield np.array(X), [y[:, i] for i in range(9)]

In [0]:
# TODO: build CNN model
# TODO: train CNN model using train_generator

In [0]:
def recall(y_true, y_pred):
    #Recall metric.

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)),axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)),axis=0)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    #Precision metric.

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)),axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)),axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    pr = precision(y_true, y_pred)
    re = recall(y_true, y_pred)
    return 2*pr*re/(pr + re)

In [38]:
base_model = MobileNetV2(include_top = False,
                   weights = 'imagenet',
                   input_shape = (IMG_SIZE, IMG_SIZE, 3),
                   pooling = 'avg')

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [40]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c

In [0]:
x = base_model.output

predictions = []
for i in range(9):
    x0 = Dense(128, activation='relu')(x)
    x0 = Dropout(0.5)(x0)
    x0 = Dense(1, activation='sigmoid')(x0)
    predictions.append(x0)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [42]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c

In [0]:
checkpoint = ModelCheckpoint('yelp_weights2.h5', save_best_only=True, verbose=1)

In [0]:
df.dropna(inplace=True)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234545 entries, 0 to 234841
Data columns (total 3 columns):
photo_id       234545 non-null int64
business_id    234545 non-null int64
labels         234545 non-null object
dtypes: int64(2), object(1)
memory usage: 7.2+ MB


In [0]:
train_df, test_df = train_test_split(df,
                                     test_size=0.01,
                                     random_state=123,
#                                     stratify=df['business_id']
                                    )
test_gen = train_generator(test_df, batch_size = len(test_df))
test_X, test_y = next(test_gen)

train_gen = train_generator(train_df)


batch_size = 100
train_gen = train_generator(train_df, batch_size = batch_size)
#steps_per_epoch = len(train_df) / batch_size

In [0]:
epochs=10
steps_per_epoch=100

In [51]:
model.fit_generator(
        train_gen,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=(test_X, test_y),
        callbacks=[checkpoint])

Epoch 1/10
100/100 [==============================] - 87s 867ms/step - loss: 4.8193 - dense_20_loss: 0.4779 - dense_22_loss: 0.5548 - dense_24_loss: 0.5230 - dense_26_loss: 0.6828 - dense_28_loss: 0.5548 - dense_30_loss: 0.4762 - dense_32_loss: 0.4226 - dense_34_loss: 0.5566 - dense_36_loss: 0.5707 - dense_20_acc: 0.7780 - dense_22_acc: 0.7108 - dense_24_acc: 0.7369 - dense_26_acc: 0.5584 - dense_28_acc: 0.7097 - dense_30_acc: 0.7657 - dense_32_acc: 0.8053 - dense_34_acc: 0.7072 - dense_36_acc: 0.7003 - val_loss: 4.9171 - val_dense_20_loss: 0.4749 - val_dense_22_loss: 0.5690 - val_dense_24_loss: 0.5272 - val_dense_26_loss: 0.6743 - val_dense_28_loss: 0.5493 - val_dense_30_loss: 0.5235 - val_dense_32_loss: 0.4816 - val_dense_34_loss: 0.5593 - val_dense_36_loss: 0.5579 - val_dense_20_acc: 0.7890 - val_dense_22_acc: 0.7063 - val_dense_24_acc: 0.7315 - val_dense_26_acc: 0.5652 - val_dense_28_acc: 0.7144 - val_dense_30_acc: 0.7634 - val_dense_32_acc: 0.7975 - val_dense_34_acc: 0.7106 - val_

100/100 [==============================] - 81s 809ms/step - loss: 4.8080 - dense_20_loss: 0.4689 - dense_22_loss: 0.5487 - dense_24_loss: 0.5244 - dense_26_loss: 0.6831 - dense_28_loss: 0.5564 - dense_30_loss: 0.4671 - dense_32_loss: 0.4313 - dense_34_loss: 0.5560 - dense_36_loss: 0.5721 - dense_20_acc: 0.7802 - dense_22_acc: 0.7210 - dense_24_acc: 0.7409 - dense_26_acc: 0.5552 - dense_28_acc: 0.7154 - dense_30_acc: 0.7733 - dense_32_acc: 0.8014 - dense_34_acc: 0.7093 - dense_36_acc: 0.7041 - val_loss: 4.8002 - val_dense_20_loss: 0.4675 - val_dense_22_loss: 0.5585 - val_dense_24_loss: 0.5305 - val_dense_26_loss: 0.6759 - val_dense_28_loss: 0.5423 - val_dense_30_loss: 0.4725 - val_dense_32_loss: 0.4360 - val_dense_34_loss: 0.5496 - val_dense_36_loss: 0.5674 - val_dense_20_acc: 0.7886 - val_dense_22_acc: 0.7204 - val_dense_24_acc: 0.7494 - val_dense_26_acc: 0.5691 - val_dense_28_acc: 0.7204 - val_dense_30_acc: 0.7783 - val_dense_32_acc: 0.8154 - val_dense_34_acc: 0.7212 - val_dense_36_ac

100/100 [==============================] - 81s 813ms/step - loss: 4.7631 - dense_20_loss: 0.4665 - dense_22_loss: 0.5463 - dense_24_loss: 0.5174 - dense_26_loss: 0.6791 - dense_28_loss: 0.5513 - dense_30_loss: 0.4710 - dense_32_loss: 0.4117 - dense_34_loss: 0.5543 - dense_36_loss: 0.5656 - dense_20_acc: 0.7846 - dense_22_acc: 0.7179 - dense_24_acc: 0.7445 - dense_26_acc: 0.5567 - dense_28_acc: 0.7163 - dense_30_acc: 0.7672 - dense_32_acc: 0.8152 - dense_34_acc: 0.7138 - dense_36_acc: 0.7151 - val_loss: 4.7995 - val_dense_20_loss: 0.4666 - val_dense_22_loss: 0.5629 - val_dense_24_loss: 0.5138 - val_dense_26_loss: 0.6754 - val_dense_28_loss: 0.5400 - val_dense_30_loss: 0.4583 - val_dense_32_loss: 0.4762 - val_dense_34_loss: 0.5453 - val_dense_36_loss: 0.5610 - val_dense_20_acc: 0.7873 - val_dense_22_acc: 0.7161 - val_dense_24_acc: 0.7540 - val_dense_26_acc: 0.5678 - val_dense_28_acc: 0.7238 - val_dense_30_acc: 0.7766 - val_dense_32_acc: 0.8107 - val_dense_34_acc: 0.7268 - val_dense_36_ac

100/100 [==============================] - 82s 816ms/step - loss: 4.7653 - dense_20_loss: 0.4682 - dense_22_loss: 0.5527 - dense_24_loss: 0.5111 - dense_26_loss: 0.6825 - dense_28_loss: 0.5484 - dense_30_loss: 0.4632 - dense_32_loss: 0.4138 - dense_34_loss: 0.5558 - dense_36_loss: 0.5696 - dense_20_acc: 0.7829 - dense_22_acc: 0.7138 - dense_24_acc: 0.7463 - dense_26_acc: 0.5568 - dense_28_acc: 0.7220 - dense_30_acc: 0.7791 - dense_32_acc: 0.8123 - dense_34_acc: 0.7128 - dense_36_acc: 0.7072 - val_loss: 4.7315 - val_dense_20_loss: 0.4586 - val_dense_22_loss: 0.5522 - val_dense_24_loss: 0.5242 - val_dense_26_loss: 0.6764 - val_dense_28_loss: 0.5349 - val_dense_30_loss: 0.4588 - val_dense_32_loss: 0.4287 - val_dense_34_loss: 0.5406 - val_dense_36_loss: 0.5571 - val_dense_20_acc: 0.7924 - val_dense_22_acc: 0.7170 - val_dense_24_acc: 0.7481 - val_dense_26_acc: 0.5708 - val_dense_28_acc: 0.7353 - val_dense_30_acc: 0.7847 - val_dense_32_acc: 0.8180 - val_dense_34_acc: 0.7310 - val_dense_36_ac

100/100 [==============================] - 81s 813ms/step - loss: 4.7367 - dense_20_loss: 0.4698 - dense_22_loss: 0.5457 - dense_24_loss: 0.5128 - dense_26_loss: 0.6815 - dense_28_loss: 0.5426 - dense_30_loss: 0.4651 - dense_32_loss: 0.4077 - dense_34_loss: 0.5496 - dense_36_loss: 0.5619 - dense_20_acc: 0.7798 - dense_22_acc: 0.7207 - dense_24_acc: 0.7424 - dense_26_acc: 0.5585 - dense_28_acc: 0.7210 - dense_30_acc: 0.7720 - dense_32_acc: 0.8117 - dense_34_acc: 0.7194 - dense_36_acc: 0.7150 - val_loss: 4.8566 - val_dense_20_loss: 0.4810 - val_dense_22_loss: 0.5551 - val_dense_24_loss: 0.5287 - val_dense_26_loss: 0.6721 - val_dense_28_loss: 0.5405 - val_dense_30_loss: 0.4745 - val_dense_32_loss: 0.4834 - val_dense_34_loss: 0.5543 - val_dense_36_loss: 0.5671 - val_dense_20_acc: 0.7941 - val_dense_22_acc: 0.7208 - val_dense_24_acc: 0.7498 - val_dense_26_acc: 0.5648 - val_dense_28_acc: 0.7246 - val_dense_30_acc: 0.7754 - val_dense_32_acc: 0.8061 - val_dense_34_acc: 0.7178 - val_dense_36_ac

100/100 [==============================] - 82s 822ms/step - loss: 4.7085 - dense_20_loss: 0.4626 - dense_22_loss: 0.5487 - dense_24_loss: 0.5093 - dense_26_loss: 0.6791 - dense_28_loss: 0.5445 - dense_30_loss: 0.4558 - dense_32_loss: 0.4000 - dense_34_loss: 0.5493 - dense_36_loss: 0.5593 - dense_20_acc: 0.7877 - dense_22_acc: 0.7264 - dense_24_acc: 0.7511 - dense_26_acc: 0.5638 - dense_28_acc: 0.7274 - dense_30_acc: 0.7788 - dense_32_acc: 0.8189 - dense_34_acc: 0.7155 - dense_36_acc: 0.7181 - val_loss: 4.7197 - val_dense_20_loss: 0.4630 - val_dense_22_loss: 0.5491 - val_dense_24_loss: 0.5035 - val_dense_26_loss: 0.6718 - val_dense_28_loss: 0.5387 - val_dense_30_loss: 0.4516 - val_dense_32_loss: 0.4370 - val_dense_34_loss: 0.5470 - val_dense_36_loss: 0.5580 - val_dense_20_acc: 0.7903 - val_dense_22_acc: 0.7272 - val_dense_24_acc: 0.7600 - val_dense_26_acc: 0.5767 - val_dense_28_acc: 0.7298 - val_dense_30_acc: 0.7890 - val_dense_32_acc: 0.8142 - val_dense_34_acc: 0.7242 - val_dense_36_ac

100/100 [==============================] - 82s 817ms/step - loss: 4.6933 - dense_20_loss: 0.4603 - dense_22_loss: 0.5460 - dense_24_loss: 0.5015 - dense_26_loss: 0.6763 - dense_28_loss: 0.5429 - dense_30_loss: 0.4527 - dense_32_loss: 0.4024 - dense_34_loss: 0.5511 - dense_36_loss: 0.5601 - dense_20_acc: 0.7878 - dense_22_acc: 0.7212 - dense_24_acc: 0.7513 - dense_26_acc: 0.5693 - dense_28_acc: 0.7225 - dense_30_acc: 0.7805 - dense_32_acc: 0.8154 - dense_34_acc: 0.7190 - dense_36_acc: 0.7116 - val_loss: 4.7582 - val_dense_20_loss: 0.4564 - val_dense_22_loss: 0.5515 - val_dense_24_loss: 0.5082 - val_dense_26_loss: 0.6710 - val_dense_28_loss: 0.5374 - val_dense_30_loss: 0.4621 - val_dense_32_loss: 0.4646 - val_dense_34_loss: 0.5509 - val_dense_36_loss: 0.5562 - val_dense_20_acc: 0.7945 - val_dense_22_acc: 0.7221 - val_dense_24_acc: 0.7609 - val_dense_26_acc: 0.5797 - val_dense_28_acc: 0.7272 - val_dense_30_acc: 0.7826 - val_dense_32_acc: 0.8095 - val_dense_34_acc: 0.7242 - val_dense_36_ac

100/100 [==============================] - 82s 823ms/step - loss: 4.6984 - dense_20_loss: 0.4590 - dense_22_loss: 0.5409 - dense_24_loss: 0.5016 - dense_26_loss: 0.6799 - dense_28_loss: 0.5469 - dense_30_loss: 0.4567 - dense_32_loss: 0.4000 - dense_34_loss: 0.5495 - dense_36_loss: 0.5640 - dense_20_acc: 0.7873 - dense_22_acc: 0.7190 - dense_24_acc: 0.7549 - dense_26_acc: 0.5691 - dense_28_acc: 0.7214 - dense_30_acc: 0.7840 - dense_32_acc: 0.8210 - dense_34_acc: 0.7142 - dense_36_acc: 0.7111 - val_loss: 4.7873 - val_dense_20_loss: 0.4831 - val_dense_22_loss: 0.5558 - val_dense_24_loss: 0.5144 - val_dense_26_loss: 0.6718 - val_dense_28_loss: 0.5391 - val_dense_30_loss: 0.4456 - val_dense_32_loss: 0.4641 - val_dense_34_loss: 0.5451 - val_dense_36_loss: 0.5682 - val_dense_20_acc: 0.7950 - val_dense_22_acc: 0.7161 - val_dense_24_acc: 0.7494 - val_dense_26_acc: 0.5776 - val_dense_28_acc: 0.7285 - val_dense_30_acc: 0.7869 - val_dense_32_acc: 0.8137 - val_dense_34_acc: 0.7234 - val_dense_36_ac

100/100 [==============================] - 83s 828ms/step - loss: 4.7132 - dense_20_loss: 0.4607 - dense_22_loss: 0.5453 - dense_24_loss: 0.5062 - dense_26_loss: 0.6798 - dense_28_loss: 0.5414 - dense_30_loss: 0.4678 - dense_32_loss: 0.4050 - dense_34_loss: 0.5457 - dense_36_loss: 0.5612 - dense_20_acc: 0.7920 - dense_22_acc: 0.7271 - dense_24_acc: 0.7561 - dense_26_acc: 0.5524 - dense_28_acc: 0.7222 - dense_30_acc: 0.7768 - dense_32_acc: 0.8194 - dense_34_acc: 0.7158 - dense_36_acc: 0.7114 - val_loss: 4.7380 - val_dense_20_loss: 0.4631 - val_dense_22_loss: 0.5463 - val_dense_24_loss: 0.5031 - val_dense_26_loss: 0.6731 - val_dense_28_loss: 0.5341 - val_dense_30_loss: 0.4533 - val_dense_32_loss: 0.4634 - val_dense_34_loss: 0.5394 - val_dense_36_loss: 0.5622 - val_dense_20_acc: 0.7950 - val_dense_22_acc: 0.7238 - val_dense_24_acc: 0.7549 - val_dense_26_acc: 0.5708 - val_dense_28_acc: 0.7225 - val_dense_30_acc: 0.7822 - val_dense_32_acc: 0.8133 - val_dense_34_acc: 0.7319 - val_dense_36_ac

100/100 [==============================] - 82s 820ms/step - loss: 4.6880 - dense_20_loss: 0.4570 - dense_22_loss: 0.5460 - dense_24_loss: 0.5025 - dense_26_loss: 0.6809 - dense_28_loss: 0.5383 - dense_30_loss: 0.4613 - dense_32_loss: 0.4056 - dense_34_loss: 0.5452 - dense_36_loss: 0.5512 - dense_20_acc: 0.7868 - dense_22_acc: 0.7232 - dense_24_acc: 0.7512 - dense_26_acc: 0.5657 - dense_28_acc: 0.7248 - dense_30_acc: 0.7795 - dense_32_acc: 0.8178 - dense_34_acc: 0.7219 - dense_36_acc: 0.7232 - val_loss: 4.7813 - val_dense_20_loss: 0.4677 - val_dense_22_loss: 0.5488 - val_dense_24_loss: 0.5158 - val_dense_26_loss: 0.6708 - val_dense_28_loss: 0.5417 - val_dense_30_loss: 0.4547 - val_dense_32_loss: 0.4663 - val_dense_34_loss: 0.5535 - val_dense_36_loss: 0.5619 - val_dense_20_acc: 0.7954 - val_dense_22_acc: 0.7259 - val_dense_24_acc: 0.7472 - val_dense_26_acc: 0.5767 - val_dense_28_acc: 0.7268 - val_dense_30_acc: 0.7873 - val_dense_32_acc: 0.8090 - val_dense_34_acc: 0.7157 - val_dense_36_ac

In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

def file_to_disk(file_name):
  from googleapiclient.http import MediaFileUpload

  file_metadata = {
   'name': file_name,
  #  'mimeType': 'text/plain'
  }
  media = MediaFileUpload(file_name, 
  #                        mimetype='text/plain',
                          resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                         media_body=media,
                                        fields='id').execute()
  print('File ID: {}'.format(created.get('id')))

In [53]:
file_to_disk('yelp_weights2.h5')

File ID: 17FAvYxEijCVekYKglwmYZRNmSX42hSFh


## Получаем предсказания

In [0]:
model.load_weights('yelp_weights2.h5')

In [0]:
class TestSequence(keras.utils.Sequence):
    """ Класс для чтения батча """
    def __init__(self, df, batch_size):
        self._df = df
        self._batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self._df) / float(self._batch_size)))

    def __getitem__(self, idx):
        result = []
        sample = self._df[idx * self._batch_size:(idx + 1) * self._batch_size]
        for _, row in sample.iterrows():
            result.append(get_image('test_photos', row['photo_id']))
        return np.array(result)

In [56]:
# читаем данные параллельно в 4потока и применяем модель
preds = model.predict_generator(
    TestSequence(test_photos_df, batch_size=100), workers=4, verbose=1)

11903/11903 [==============================] - 6780s 570ms/step


In [0]:
test_photos_df['labels'] = [[float(preds[j][i]) for j in range(9)] 
                            for i in range(len(test_photos_df))]

In [58]:
test_photos_df.head()

,photo_id,business_id,labels
0,317818,003sg,"[0.14658589661121368, 0.4474533200263977, 0.65..."
1,30679,003sg,"[0.2607298493385315, 0.4288123846054077, 0.566..."
2,455084,003sg,"[0.047367196530103683, 0.6503982543945312, 0.8..."
3,371381,003sg,"[0.13845843076705933, 0.6566969752311707, 0.62..."
4,86224,003sg,"[0.0950169712305069, 0.7989132404327393, 0.893..."


In [0]:
business_df = test_photos_df.groupby('business_id')

In [0]:
submission = []
for k, labels in business_df['labels'].apply(list).iteritems():
    labels = np.array(labels).mean(axis=0)
    labels = np.where(labels > 0.5)[0]
    submission.append((k, ' '.join(map(str, labels))))

In [0]:
submission_df = pd.DataFrame(submission, columns=('business_id', 'labels'))
submission_df.to_csv('submission_yelp.csv', index=False)


In [62]:
file_to_disk('submission_yelp.csv')

File ID: 1qlhS3Z-I5qR1SCcIyXYlEyTArb00yqKG


In [0]:
#public scode 0.70228